In [ ]:
import os
import pandas as pd
import numpy as np  # make sure to add
import requests
from bs4 import BeautifulSoup
import logging  # make sure to add
import calendar  # make sure to add
from headers import headers  # change to your own headers file or add variable in code
from edgar_functions import *
import json
#tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN'] 

In [ ]:

#data = pd.read_csv('tickers.csv', header=None)
#tickers = data.values.T[0].tolist()
#ticker = ticker[60:64]
#tickers

## Second try 

In [ ]:
def get_statements_for_tickers(tickers, statement_name):
    """
    Retrieves financial statements for multiple tickers.

    Args:
        tickers (list): List of stock ticker symbols.
        statement_name (str): Name of the financial statement.

    Returns:
        pd.DataFrame: Concatenated DataFrame containing financial statements for all tickers.
    """
    dfs = []  # List to store DataFrames for each ticker
    for ticker in tickers:
        try:
            # Get accession number
            accn = get_filtered_filings(ticker, ten_k=False, just_accession_numbers=True, headers=headers)
            acc_num = accn.iloc[0].replace('-', '')

            # Process statement
            df = process_one_statement(ticker, acc_num, statement_name)
            
            # Append DataFrame to list with ticker column
            if df is not None:
                df['Ticker'] = ticker  # Add ticker column
                dfs.append(df)
            else:
                print(f"Failed to process statement for ticker: {ticker}")
        except Exception as e:
            print(f"Error processing ticker {ticker}: {e}")

    # Concatenate DataFrames for all tickers
    if len(dfs) > 0:
        return pd.concat(dfs)
    else:
        return None

# Example usage:
tickers = ['DAL']  # Example list of tickers
statement_name ="cash_flow_statement" #"income_statement" #'balance_sheet'  # Example statement name
result_df = get_statements_for_tickers(tickers, statement_name)

result_df.head()


In [ ]:

result_df = result_df.reset_index()
result_df[['taxonomy', 'fact_taxonomy']] = result_df['index'].str.split('_', expand=True)

In [ ]:
result_df

In [ ]:
#result_df= result_df.iloc[:,2:]
result_df

In [ ]:
import re
def split_camel_case(s):
    return ' '.join(re.findall(r'[A-Z][a-z]*', s))

# Apply function to 'fact_taxonomy' column
result_df['fact_taxonomy'] = result_df['fact_taxonomy'].apply(split_camel_case)
result_df = result_df.drop('index', axis = 1)

In [ ]:
# Reverse the order of columns
reversed_columns = list(reversed(result_df.columns))

# Reorder columns using reindex
result_df = result_df.reindex(columns=reversed_columns)

# Print or use the DataFrame with reversed columns

result_df

In [ ]:
columns = ['Ticker'] + [col for col in result_df.columns if col != 'Ticker']
result_df = result_df[columns]

# Print the combined DataFrame with 'Ticker' as the first column
#combined_dfs= combined_df
result_df

In [ ]:
result_df.to_csv('MSFT_CF.csv')

In [ ]:
#####